## 라이브러리

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

## 데이터 불러오기

In [2]:
import pandas as pd
#데이터 로드
X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv")
y_test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv')

display(X_train.head())
display(y_train.head())

,StudentID,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,1714,GP,F,18,U,GT3,T,4,3,other,...,no,4,3,3,1,1,3,0,14,13
1,1254,GP,F,17,U,GT3,T,4,3,health,...,yes,4,4,3,1,3,4,0,13,15
2,1639,GP,F,16,R,GT3,T,4,4,health,...,no,2,4,4,2,3,4,6,10,11
3,1118,GP,M,16,U,GT3,T,4,4,services,...,no,5,3,3,1,3,5,0,15,13
4,1499,GP,M,19,U,GT3,T,3,2,services,...,yes,4,5,4,1,1,4,0,5,0


,StudentID,G3
0,1714,14
1,1254,15
2,1639,11
3,1118,13
4,1499,0


## 1. 데이터셋 확인, EDA

### shape

In [3]:
print(X_train.shape)
print("-"*80)
print(X_test.shape)
print("-"*80)
print(y_train.shape)
print("-"*80)

(678, 33)
--------------------------------------------------------------------------------
(366, 33)
--------------------------------------------------------------------------------
(678, 2)
--------------------------------------------------------------------------------


### info

In [4]:
print(X_train.info())
print("-"*80)
print(X_test.info())
print("-"*80)
print(y_train.info())
print("-"*80)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   StudentID   678 non-null    int64 
 1   school      678 non-null    object
 2   sex         678 non-null    object
 3   age         678 non-null    int64 
 4   address     678 non-null    object
 5   famsize     678 non-null    object
 6   Pstatus     678 non-null    object
 7   Medu        678 non-null    int64 
 8   Fedu        678 non-null    int64 
 9   Mjob        678 non-null    object
 10  Fjob        678 non-null    object
 11  reason      678 non-null    object
 12  guardian    678 non-null    object
 13  traveltime  678 non-null    int64 
 14  studytime   678 non-null    int64 
 15  failures    678 non-null    int64 
 16  schoolsup   678 non-null    object
 17  famsup      678 non-null    object
 18  paid        678 non-null    object
 19  activities  678 non-null    object
 20  nursery   

### isna

In [5]:
print(X_train.isna().sum().sum())
print("-"*80)
print(X_test.isna().sum().sum())
print("-"*80)
print(y_train.isna().sum().sum())
print("-"*80)

0
--------------------------------------------------------------------------------
0
--------------------------------------------------------------------------------
0
--------------------------------------------------------------------------------


### 불필요 컬럼 제거

In [10]:
# X_test_ID = X_test['StudentID']
# X_train.drop(columns=['StudentID'], inplace=True)
# X_test.drop(columns=['StudentID'], inplace=True)
# y_train.drop(columns=['StudentID'], inplace=True)

KeyError: 'StudentID'

## 2. Feature Engineering

### 표준화(스케일링)

In [11]:
from sklearn.preprocessing import MinMaxScaler
Scaler = StandardScaler()
X_train[['age', 'G1', 'G2']] = Scaler.fit_transform(X_train[['age', 'G1', 'G2']])
X_test[['age', 'G1', 'G2']] = Scaler.fit_transform(X_test[['age', 'G1', 'G2']])

### 원-핫 인코딩(pd.get_dummies)

In [12]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,1.033310,4,3,1,4,0,4,3,3,1,...,1,0,0,1,0,1,0,1,1,0
1,0.226185,4,3,1,3,0,4,4,3,1,...,0,1,0,1,0,1,0,1,0,1
2,-0.580939,4,4,1,2,0,2,4,4,2,...,0,1,0,1,0,1,1,0,1,0
3,-0.580939,4,4,1,3,0,5,3,3,1,...,0,1,0,1,0,1,0,1,1,0
4,1.840434,3,2,1,1,3,4,5,4,1,...,1,0,0,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,-1.388064,4,4,1,1,0,5,3,3,1,...,0,1,1,0,0,1,0,1,1,0
674,-1.388064,4,4,4,4,0,1,3,5,3,...,0,1,0,1,0,1,0,1,0,1
675,0.226185,4,4,2,1,0,4,1,1,2,...,1,0,0,1,0,1,0,1,1,0
676,0.226185,1,1,2,1,0,4,5,5,1,...,0,1,1,0,0,1,0,1,0,1


### 데이터 분류(train_test_split)

In [13]:
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X_train, y_train, test_size=0.3)

## 3. 모델링

### 랜덤포레스트

In [19]:
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor(500, max_depth = 5).fit(X_tr, y_tr)
modelRF.score(X_ts, y_ts)

C:\Users\inseo\AppData\Local\Temp\ipykernel_18496\3273434341.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelRF = RandomForestRegressor(500, max_depth = 5).fit(X_tr, y_tr)


0.8788407193862537

## 4. 모델 학습 및 예측, 평가

In [26]:
from sklearn.metrics import r2_score
y_pred = modelRF.predict(X_ts)
print('RF r2_score', r2_score(y_ts, y_pred))

ValueError: multi_class must be in ('ovo', 'ovr')